In [1]:
# https://github.com/evanm31/p4k-scraper/blob/master/data/scrapefork.py

In [2]:
# !python -m pip install retry
# !python -m pip install backoff

Import packages and set uip enviornment

In [3]:
import urllib
import re
import time
import retry
import backoff
import pandas as pd
import requests
import datetime
import os
from bs4 import BeautifulSoup

import pitchfork_ratings_def # my custom functions from github

if os.name == "posix":
    data_path = "/Users/kylezengo/Desktop/DS/Music Ratings/"

Load historic data

In [4]:
# Load list of album review links
links_file_list = os.listdir(data_path+"links")

links_list = []
for i in links_file_list:
    links = open(data_path+"links/"+i)
    links = links.read().splitlines()
    
    links_list += links

links_list = list(set(links_list))
links_list = [x for x in links_list if x.startswith("https")]
print(f'{len(links_list)} album review links')

26801 album review links


In [5]:
# # Not sure what to do with this yet...
# pd.read_csv(data_path+"reviews/pitchfork_reviews_hist.csv").head()

In [6]:
reviews_file_list = os.listdir(data_path+"reviews")
reviews_file_list.remove('pitchfork_reviews_hist.csv')

reviews = []
for i in reviews_file_list:
    df = pd.read_csv(data_path+"reviews/"+i)
    reviews.append(df)

reviews = pd.concat(reviews).drop_duplicates().reset_index(drop=True)
print(f'{reviews.shape[0]} album reviews in data frame')
reviews.head()

9980 album reviews in data frame


,artist,album,score,genre,review,best,reviewed_date,link
0,Dan Deacon,Gliss Riffer,7.2,Electronic,It's been almost a decade since Baltimore sing...,0,"February 23, 2015",https://pitchfork.com/reviews/albums/20279-gli...
1,Born Under a Rhyming Planet,Diagonals,7.5,Electronic / Rock,"Despite the name, Demdike Stare’s DDS is no va...",0,"September 8, 2022",https://pitchfork.com/reviews/albums/born-unde...
2,BabyTron,6,6.8,Rap,BabyTron likes to goof around. On his early sc...,0,"June 15, 2023",https://pitchfork.com/reviews/albums/babytron-6/
3,Thuja,Ghost Plants,7.9,NaN,Thuja are a quartet based out of San Francisco...,0,"April 10, 2002",https://pitchfork.com/reviews/albums/8050-ghos...
4,Ivory Coast,Clouds,6.6,NaN,"Life is evanescent and puzzling, seldom sittin...",0,"September 20, 2001",https://pitchfork.com/reviews/albums/4142-clouds/


In [ ]:
# do list for links n

Get the link of each album review and save off to text file

In [7]:
%%time
# 3-4s per page (5-10min to run)

new_links = []
i = 1
while requests.get("https://pitchfork.com/reviews/albums/?page="+str(i)).status_code==200:
    req = urllib.request.Request("https://pitchfork.com/reviews/albums/?page=" + str(i))
    html_page = pitchfork_ratings_def.urlopen_with_retry(req)
    print(f'{i}: html_page status code {html_page.getcode()}')

    soup = BeautifulSoup(html_page, "lxml")
    
    for link in soup.findAll('a'):
        link_get_href = link.get('href')
        if link_get_href == '/reviews/albums/':
            pass
        elif link_get_href.startswith( '/reviews/albums/?genre=' ):
            pass
        elif link_get_href.startswith( '/reviews/albums/' ):
            new_links.append("https://pitchfork.com"+link_get_href)
    i+=1

new_links = list(set(new_links))

1: html_page status code 200
2: html_page status code 200
3: html_page status code 200
4: html_page status code 200
5: html_page status code 200
6: html_page status code 200
7: html_page status code 200
8: html_page status code 200
9: html_page status code 200
10: html_page status code 200
11: html_page status code 200
12: html_page status code 200
13: html_page status code 200
14: html_page status code 200
15: html_page status code 200
16: html_page status code 200
17: html_page status code 200
18: html_page status code 200
19: html_page status code 200
20: html_page status code 200
21: html_page status code 200
22: html_page status code 200
23: html_page status code 200
24: html_page status code 200
25: html_page status code 200
26: html_page status code 200
27: html_page status code 200
28: html_page status code 200
29: html_page status code 200
30: html_page status code 200
31: html_page status code 200
32: html_page status code 200
33: html_page status code 200
34: html_page statu

In [8]:
len(new_links)

9981

In [9]:
# Save list since takes time to generate
with open(f'{data_path}links/new_links_{datetime.datetime.now().date()}.txt', 'w') as f:
    for item in new_links:
        f.write("%s\n" % item)

Scrape reviews for each album and save to csv

In [10]:
links_list_with_new = links_list+new_links
    
checked_links = set(df['link'])
not_checked_links = [x for x in links_list_with_new if x not in checked_links]
print(f'{len(not_checked_links)} links without review data')

29851 links without review data


In [11]:
# # test links
# new_links = ["https://pitchfork.com/reviews/albums/james-devane-searching/",
#              "https://pitchfork.com/reviews/albums/blood-incantation-absolute-elsewhere/",
#              "https://pitchfork.com/reviews/albums/body-meat-starchris/",
#              "https://pitchfork.com/reviews/albums/fred-again-actual-life-3-january-1-september-9-2022/",
#              "https://pitchfork.com/reviews/albums/100-gecs-10000-gecs/"]

In [12]:
%%time
# 2s? per page (long time to run >1 hr)

dat = []
for idx, i in enumerate(not_checked_links):
    print(f'{idx}: {i}')
    dat.append(pitchfork_ratings_def.gather_info(i))

new_reviews = pd.concat(dat).reset_index(drop=True)

0: https://pitchfork.com/reviews/albums/9491-lion-devours-the-sun/
1: https://pitchfork.com/reviews/albums/tyler-childers-rustin-in-the-rain/
2: https://pitchfork.com/reviews/albums/shygirl-alias-ep/
3: https://pitchfork.com/reviews/albums/19824-drexciya-transllusion-neptunes-lairthe-opening-of-the-cerebral-gate/
4: https://pitchfork.com/reviews/albums/14152-the-outsiders-are-back/
5: https://pitchfork.com/reviews/albums/nicolas-godin-concrete-and-glass/
6: https://pitchfork.com/reviews/albums/pinegrove-skylight/
7: https://pitchfork.com/reviews/albums/13461-colonial-drones/
8: https://pitchfork.com/reviews/albums/19434-craig-leon-anthology-of-interplanetary-folk-music-vol-1-nommosvisiting/
9: https://pitchfork.com/reviews/albums/10384-lets-drag-our-feet/
10: https://pitchfork.com/reviews/albums/17858-james-ferraro-cold/
11: https://pitchfork.com/reviews/albums/20135-girls-in-peacetime-want-to-dance/
12: https://pitchfork.com/reviews/albums/12504-dark-end-of-the-street-ep/
13: https://

120: https://pitchfork.com/reviews/albums/15450-suck-it-and-see/
121: https://pitchfork.com/reviews/albums/6641-96-drum-n-bass-classixxx/
122: https://pitchfork.com/reviews/albums/hope-sandoval-and-the-warm-inventions-son-of-a-lady-ep/
123: https://pitchfork.com/reviews/albums/14757-bloody-war-songs-1924-1939-the-battles-rage-on/
124: https://pitchfork.com/reviews/albums/margo-price-thats-how-rumors-get-started/
125: https://pitchfork.com/reviews/albums/16197-gonjasufi-muzzle/
126: https://pitchfork.com/reviews/albums/12088-dj-drama-presents-the-preview/
127: https://pitchfork.com/reviews/albums/18294-the-love-language-ruby-red/
128: https://pitchfork.com/reviews/albums/12841-beauties-never-die/
129: https://pitchfork.com/reviews/albums/19456-martyn-the-air-between-words/
130: https://pitchfork.com/reviews/albums/lil-peep-come-over-when-youre-sober-pt-2/
131: https://pitchfork.com/reviews/albums/10911-elegies-to-lessons-learnt/
132: https://pitchfork.com/reviews/albums/356-audioslave/


234: https://pitchfork.com/reviews/albums/12752-dirtbox-ep-when-machines-exceed-human-intelligence/
235: https://pitchfork.com/reviews/albums/15998-parallax/
236: https://pitchfork.com/reviews/albums/22505-holiday-for-strings/
237: https://pitchfork.com/reviews/albums/18746-noveller-no-dreams/
238: https://pitchfork.com/reviews/albums/migos-culture-iii/
239: https://pitchfork.com/reviews/albums/3510-truly-she-is-none-other/
240: https://pitchfork.com/reviews/albums/9562-paper-tigers/
241: https://pitchfork.com/reviews/albums/9172-in-bocca-al-lupo/
242: https://pitchfork.com/reviews/albums/octavian-spaceman/
243: https://pitchfork.com/reviews/albums/9813-find-shelter/
244: https://pitchfork.com/reviews/albums/8619-illuminated-by-the-light/
245: https://pitchfork.com/reviews/albums/11440-films/
246: https://pitchfork.com/reviews/albums/young-nudy-anyways/
247: https://pitchfork.com/reviews/albums/doug-paisley-starter-home/
248: https://pitchfork.com/reviews/albums/big-thief-ufof/
249: ht

354: https://pitchfork.com/reviews/albums/17284-furniture/
355: https://pitchfork.com/reviews/albums/14402-get-him-to-the-greek-ost/
356: https://pitchfork.com/reviews/albums/15685-satin-panthers-ep/
357: https://pitchfork.com/reviews/albums/9149-come-into-our-house/
358: https://pitchfork.com/reviews/albums/14043-white-mystery/
359: https://pitchfork.com/reviews/albums/1336-twinkle-echo/
360: https://pitchfork.com/reviews/albums/the-sisters-of-mercy-floodland/
361: https://pitchfork.com/reviews/albums/10860-to-all-dead-sailors/
362: https://pitchfork.com/reviews/albums/gil-scott-heron-pieces-of-a-man/
363: https://pitchfork.com/reviews/albums/7823-four-winds-the-walker/
364: https://pitchfork.com/reviews/albums/16406-tanlines-mixed-emotions/
365: https://pitchfork.com/reviews/albums/11444-hnic-pt-2/
366: https://pitchfork.com/reviews/albums/9469-release-therapy/
367: https://pitchfork.com/reviews/albums/alex-lahey-the-answer-is-always-yes/
368: https://pitchfork.com/reviews/albums/673

470: https://pitchfork.com/reviews/albums/6111-trans-balkan-express/
471: https://pitchfork.com/reviews/albums/2651-kingdom-of-champions/
472: https://pitchfork.com/reviews/albums/16394-daniel-rossen-silent-hourgolden-mile-ep/
473: https://pitchfork.com/reviews/albums/3961-the-devil-isnt-red/
474: https://pitchfork.com/reviews/albums/18617-skeletonwitch-serpents-unleashed/
475: https://pitchfork.com/reviews/albums/19748-the-history-of-apple-pie-feel-something/
476: https://pitchfork.com/reviews/albums/17353-nebula-dance/
477: https://pitchfork.com/reviews/albums/14116-thing/
478: https://pitchfork.com/reviews/albums/2136-interstella-5555-the-5tory-of-the-5ecret-5tar-5ystem/
479: https://pitchfork.com/reviews/albums/frances-quinlan-likewise/
480: https://pitchfork.com/reviews/albums/the-blow-brand-new-abyss/
481: https://pitchfork.com/reviews/albums/10357-collective-psychosis-begone/
482: https://pitchfork.com/reviews/albums/bisk-vacation-package/
483: https://pitchfork.com/reviews/albu

...

29383: https://pitchfork.com/reviews/albums/katie-von-schleicher-consummation/
29384: https://pitchfork.com/reviews/albums/matt-ox-ox/
29385: https://pitchfork.com/reviews/albums/magdalena-bay-mercurial-world/
29386: https://pitchfork.com/reviews/albums/eluvium-pianoworks/
29387: https://pitchfork.com/reviews/albums/roy-ayers-adrian-younge-ali-shaheed-roy-ayers-jid-002/
29388: https://pitchfork.com/reviews/albums/cusp-you-can-do-it-all/
29389: https://pitchfork.com/reviews/albums/22228-crappin-you-negative/
29390: https://pitchfork.com/reviews/albums/suns-signature-suns-signature/
29391: https://pitchfork.com/reviews/albums/the-caretaker-everywhere-at-the-end-of-time-stage-4/
29392: https://pitchfork.com/reviews/albums/nicolas-jaar-telas/
29393: https://pitchfork.com/reviews/albums/narrow-head-12th-house-rock/
29394: https://pitchfork.com/reviews/albums/ornette-coleman-round-trip-ornette-coleman-on-blue-note/
29395: https://pitchfork.com/reviews/albums/weakened-friends-common-blah/
293

29495: https://pitchfork.com/reviews/albums/li-daiguo-xiao-gong/
29496: https://pitchfork.com/reviews/albums/teddy3-lillyanna/
29497: https://pitchfork.com/reviews/albums/klein-touched-by-an-angel/
29498: https://pitchfork.com/reviews/albums/stella-donnelly-flood/
29499: https://pitchfork.com/reviews/albums/rp-boo-legacy-volume-2/
29500: https://pitchfork.com/reviews/albums/nathan-salsburg-psalms/
29501: https://pitchfork.com/reviews/albums/futuro-conjunto-futuro-conjunto/
29502: https://pitchfork.com/reviews/albums/sour-widows-crossing-over-ep/
29503: https://pitchfork.com/reviews/albums/phoenix-alpha-zulu/
29504: https://pitchfork.com/reviews/albums/father-awful-swim/
29505: https://pitchfork.com/reviews/albums/wand-perfume-ep/
29506: https://pitchfork.com/reviews/albums/rjmrla-dj-mustard-the-ghetto/
29507: https://pitchfork.com/reviews/albums/koloah-serenity/
29508: https://pitchfork.com/reviews/albums/22182-michal-turtle-phantoms-of-dreamland/
29509: https://pitchfork.com/reviews/a

29604: https://pitchfork.com/reviews/albums/23145-the-wild/
29605: https://pitchfork.com/reviews/albums/fire-toolz-eternal-home/
29606: https://pitchfork.com/reviews/albums/laurie-anderson-tenzin-choegyal-jess-songs-from-the-bardo/
29607: https://pitchfork.com/reviews/albums/1010benja-ten-total/
29608: https://pitchfork.com/reviews/albums/22494-building-a-beginning/
29609: https://pitchfork.com/reviews/albums/21723-congrats/
29610: https://pitchfork.com/reviews/albums/21030-utakata-no-hibi/
29611: https://pitchfork.com/reviews/albums/julien-baker-b-sides-ep/
29612: https://pitchfork.com/reviews/albums/steve-mason-about-the-light/
29613: https://pitchfork.com/reviews/albums/23301-hopeless-fountain-kingdom/
29614: https://pitchfork.com/reviews/albums/the-streets-the-darker-the-shadow-the-brighter-the-light/
29615: https://pitchfork.com/reviews/albums/haviah-mighty-13th-floor/
29616: https://pitchfork.com/reviews/albums/22576-qtt4/
29617: https://pitchfork.com/reviews/albums/nef-the-phara

29713: https://pitchfork.com/reviews/albums/the-lemon-twigs-songs-for-the-general-public/
29714: https://pitchfork.com/reviews/albums/sunn-o-life-metal/
29715: https://pitchfork.com/reviews/albums/molchat-doma-belaya-polosa/
29716: https://pitchfork.com/reviews/albums/babychiefdoit-animals-only/
29717: https://pitchfork.com/reviews/albums/23189-untitled/
29718: https://pitchfork.com/reviews/albums/the-gerbils-are-you-sleepy/
29719: https://pitchfork.com/reviews/albums/morly-til-i-start-speaking/
29720: https://pitchfork.com/reviews/albums/mount-eerie-now-only/
29721: https://pitchfork.com/reviews/albums/22810-myths-002-ep/
29722: https://pitchfork.com/reviews/albums/21177-music-for-life-itself-the-interrupters/
29723: https://pitchfork.com/reviews/albums/the-armed-perfect-saviors/
29724: https://pitchfork.com/reviews/albums/boris-no/
29725: https://pitchfork.com/reviews/albums/catherine-christer-hennix-selected-early-keyboard-works/
29726: https://pitchfork.com/reviews/albums/ghostface

29827: https://pitchfork.com/reviews/albums/sam-smith-love-goes/
29828: https://pitchfork.com/reviews/albums/eddie-vedder-earthling/
29829: https://pitchfork.com/reviews/albums/don-toliver-life-of-a-don/
29830: https://pitchfork.com/reviews/albums/wednesday-rat-saw-god/
29831: https://pitchfork.com/reviews/albums/bob-weir-ace-50th-anniversary-deluxe-edition/
29832: https://pitchfork.com/reviews/albums/goose-dripfield/
29833: https://pitchfork.com/reviews/albums/taylor-swift-fearless-taylors-version/
29834: https://pitchfork.com/reviews/albums/dot-allison-heart-shaped-scars/
29835: https://pitchfork.com/reviews/albums/anycia-princess-pop-that/
29836: https://pitchfork.com/reviews/albums/kindness-something-like-a-war/
29837: https://pitchfork.com/reviews/albums/lindstrom-everyone-else-is-a-stranger/
29838: https://pitchfork.com/reviews/albums/mr-greg-cass-mccombs-mr-greg-and-cass-mccombs-sing-and-play-new-folk-songs-for-children/
29839: https://pitchfork.com/reviews/albums/chief-keef-zay

In [13]:
# new_reviews = pd.concat(dat).reset_index(drop=True)
# new_reviews

In [14]:
# # Testing errors
# album_link = "https://pitchfork.com/reviews/albums/various-artists-even-the-forest-hums-ukrainian-sonic-archives/" 
# gather_info(album_link)

# page = requests.get(album_link) #request URL
# soup = BeautifulSoup(page.content, 'html.parser') #parse with beautifulsoup
# soup

In [15]:
new_reviews.to_csv(f'{data_path}reviews/new_reviews_{datetime.datetime.now().date()}.csv',index=False)